In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai import *
from fastai.text import *


In [3]:
#torch.cuda.set_device(0)

In [4]:
# language models can use a lot of GPU, 
#so you may need to decrease batchsize here

bs = 48

In [5]:
#preparing the data (on a sample)

path = untar_data(URLs.IMDB_SAMPLE)
path.ls()

[PosixPath('/home/odemakinde/.fastai/data/imdb_sample/texts.csv')]

In [6]:
data_lm = TextDataBunch.from_csv(path, 'texts.csv')

In [7]:
# Tokenization
# split raw sentences into words, or more exactly tokens.
# we need to take care of punctuations
# some words are contractions of two different words, like isn't or don't
# we may need to clean some parts of our texts. if there's HTML code for instance.



In [8]:
# numericalization

# once we have extracted tokens from our text, we convert
# to integers by creating a list of all the words used. 
# we only keep the ones that appear at list twice with a maximum
# vocabulary size of 60,000 (by default) and replace the ones
# that don't make the cut by the unknown token UNK.


In [10]:
data_lm.vocab.itos[:10]

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 'the']

In [11]:
len(data_lm.vocab.itos)

8688

In [19]:
sample = data_lm.train_ds[0][0]
sample

Text xxbos xxmaj there is n't a whole lot going on in this story -- just two men xxunk very different ways of xxunk memories of xxmaj vietnam . xxmaj but what it lacks in premise , it more than makes up for in acting and realism . xxmaj it 's a quiet film about the bonds of friendship and shared experience . xxmaj we even get romance ( not gratuitous -- just another very real piece of this story ) . xxmaj it 's well worth seeing .

In [22]:
print(data_lm.train_ds[0][0].data[:10])
len(data_lm.train_ds[0][0].data)

[  2   5  56  16  37  13 233 213 180  36]


88

In [24]:
# using data block API with NLP

data = (TextList.from_csv(path, 'texts.csv', cols = 'text')
       .split_from_df(col = 2)
       .label_from_df(cols = 0)
       .databunch())


In [27]:
# language model
path = untar_data(URLs.IMDB)
path.ls()

[PosixPath('/home/odemakinde/.fastai/data/imdb/imdb.vocab'),
 PosixPath('/home/odemakinde/.fastai/data/imdb/tmp_lm'),
 PosixPath('/home/odemakinde/.fastai/data/imdb/unsup'),
 PosixPath('/home/odemakinde/.fastai/data/imdb/test'),
 PosixPath('/home/odemakinde/.fastai/data/imdb/tmp_clas'),
 PosixPath('/home/odemakinde/.fastai/data/imdb/train'),
 PosixPath('/home/odemakinde/.fastai/data/imdb/README')]

In [28]:
(path/'train').ls()

[PosixPath('/home/odemakinde/.fastai/data/imdb/train/neg'),
 PosixPath('/home/odemakinde/.fastai/data/imdb/train/labeledBow.feat'),
 PosixPath('/home/odemakinde/.fastai/data/imdb/train/pos'),
 PosixPath('/home/odemakinde/.fastai/data/imdb/train/unsupBow.feat')]

In [29]:
# WikiText-103 pretrained model
#its a dataset (a pre-trained language model)

In [31]:
# creating the TextLMDataBunch

df = (TextList.from_folder(path)
     #Inputs: all the text files in path
     .filter_by_folder(include = ['train','test'])
     .random_split_by_pct(0.1)
     .label_for_lm()
     .databunch(bs =bs))

/home/odemakinde/anaconda3/lib/python3.7/site-packages/fastai/data_block.py:215: UserWarning: `random_split_by_pct` is deprecated, please use `split_by_rand_pct`.
  warn("`random_split_by_pct` is deprecated, please use `split_by_rand_pct`.")


In [32]:
len(data_lm.vocab.itos)

8688

In [33]:
data_lm.show_batch()

text,target
"xxbos xxmaj raising xxmaj victor xxmaj vargas : a xxmaj review \n \n xxmaj you know , xxmaj raising xxmaj victor xxmaj vargas is like sticking your hands into a big , steaming bowl of xxunk . xxmaj it 's warm and gooey , but you 're not sure if it feels right . xxmaj try as i might , no matter how warm and gooey xxmaj raising xxmaj",negative
"xxbos xxup the xxup shop xxup around xxup the xxup corner is one of the sweetest and most feel - good romantic comedies ever made . xxmaj there 's just no getting around that , and it 's hard to actually put one 's feeling for this film into words . xxmaj it 's not one of those films that tries too hard , nor does it come up with",positive
"xxbos xxmaj now that xxmaj che(2008 ) has finished its relatively short xxmaj australian cinema run ( extremely limited xxunk screen in xxmaj sydney , after xxunk ) , i can xxunk join both xxunk of "" xxmaj at xxmaj the xxmaj movies "" in taking xxmaj steven xxmaj soderbergh to task . \n \n xxmaj it 's usually satisfying to watch a film director change his style /",negative
"xxbos xxmaj many neglect that this is n't just a classic due to the fact that it 's the first xxup 3d game , or even the first xxunk - up . xxmaj it 's also one of the first stealth games , one of the xxunk definitely the first ) truly claustrophobic games , and just a pretty well - xxunk gaming experience in general . xxmaj with graphics",positive
"xxbos i really wanted to love this show . i truly , honestly did . \n \n xxmaj for the first time , gay viewers get their own version of the "" xxmaj the xxmaj bachelor "" . xxmaj with the help of his obligatory "" hag "" xxmaj xxunk , xxmaj james , a good looking , well - to - do thirty - something has the chance",negative


In [34]:
data_lm.save('lm_databunch')

In [37]:
# loading saved data
#data_lm = load_data(path, 'lm_databunch', bs = bs)


In [41]:
#learn_lm = language_model_learner(data_lm, AWD_LSTM, drop_mult= 0.3)

#wiki_itos = pickle.load(open(Config().model_path()/'wt103-1/itos_wt103.pkl','rb'))

In [42]:
#wiki_itos[:10]

In [43]:
vocab = data_lm.vocab

In [44]:
vocab.stoi['stingray']

0

In [46]:
from scipy.spatial.distance import cosine as dist
#enc = learn_lm.model[0].encoder
#enc.weight.size()